### Импорт необходимых библиотек

 Импортируем библиотеки для работы с данными (`pandas`, `numpy`), сериализации (`json`, `joblib`) и предобработки признаков (`VarianceThreshold`, `MinMaxScaler`) из библиотеки `scikit-learn`.


In [1]:
import pandas as pd
import numpy as np
import json

from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import MinMaxScaler
import joblib

### Загрузка обучающих дескрипторов

Загружаем различные типы дескрипторов для обучающей выборки по индексному столбцу `CASRN`:
- ECFP6 в битовом и количественном представлении
- MACCS ключи
- RDKit 2D дескрипторы
- Mordred дескрипторы


In [2]:
train_ecfp6_bits = pd.read_csv('../data/descriptors/train_ecfp6_bits.csv', index_col='CASRN')
train_ecfp6_counts = pd.read_csv('../data/descriptors/train_ecfp6_counts.csv', index_col='CASRN')
train_maccs = pd.read_csv('../data/descriptors/train_maccs.csv', index_col='CASRN')
train_rdkit2d = pd.read_csv('../data/descriptors/train_rdkit2d.csv', index_col='CASRN')
train_mordred = pd.read_csv('../data/descriptors/train_mordred.csv', index_col='CASRN')

### Функция для отбора признаков

Определяем функцию `feature_selection`, которая удаляет признаки с нулевой дисперсией и высоко коррелированные признаки (порог по умолчанию — 0.95). Это помогает уменьшить размерность и избежать мультиколлинеарности.


In [3]:
def feature_selection(df, nonzero_thrd = 0.0, cor_thrd = 0.95):

    selector = VarianceThreshold(nonzero_thrd)
    selector.fit(df)
    nonzero_df = df[df.columns[selector.get_support(indices=True)]]
    corr_matrix = nonzero_df.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > cor_thrd)]
    
    return nonzero_df.drop(nonzero_df[to_drop], axis=1)

### Применение отбора признаков к обучающим данным

Применяем функцию `feature_selection` ко всем обучающим дескрипторам и выводим размеры отфильтрованных наборов признаков.


In [4]:
filtered_train_ecfp6_bits = feature_selection(train_ecfp6_bits)
filtered_train_ecfp6_counts = feature_selection(train_ecfp6_counts)
filtered_train_maccs = feature_selection(train_maccs)
filtered_train_rdkit2d = feature_selection(train_rdkit2d)
filtered_train_mordred = feature_selection(train_mordred)

c:\Users\PC\anaconda3\Lib\site-packages\numpy\lib\nanfunctions.py:1746: RuntimeWarning: overflow encountered in multiply
  sqr = np.multiply(arr, arr, out=arr, where=where)


In [5]:
filtered_train_ecfp6_bits.shape, filtered_train_ecfp6_counts.shape,filtered_train_maccs.shape

((8994, 2048), (8994, 1584), (8994, 148))

In [6]:
filtered_train_mordred.shape, filtered_train_rdkit2d.shape

((8994, 446), (8994, 164))

Создаём словарь `dict_features`, в котором для каждого типа дескрипторов сохраняем список имён признаков, прошедших отбор. Сохраняем этот словарь в JSON-файл для последующего использования при обработке тестовых данных.


In [7]:
desc = ['ecfp6_bits', 'ecfp6_counts', 'maccs', 'rdkit2d', 'mordred']
filtered_features = [
    list(filtered_train_ecfp6_bits),
    list(filtered_train_ecfp6_counts),
    list(filtered_train_maccs),
    list(filtered_train_rdkit2d),
    list(filtered_train_mordred),
]

dict_features = dict(zip(desc, filtered_features))

### Сохранение отобранных признаков

In [8]:
with open('../data/descriptors/filtered_features.json', 'w') as f:
    json.dump(dict_features, f)


### Нормализация числовых признаков

Определяем две функции:
- `feature_norm_fit` — выполняет масштабирование признаков обучающей выборки в диапазон [0, 1] и возвращает обученный `MinMaxScaler`. 

- `feature_norm_transform` — масштабирует тестовые признаки с использованием уже обученного скейлера.
Нормализация применяется только к дескрипторам `rdkit2d` и `mordred`.


In [9]:
def feature_norm_fit(train_df , scaler = MinMaxScaler()):
    array =  train_df.values
    df_norm = pd.DataFrame(scaler.fit_transform(array), columns=train_df.columns, index=train_df.index)
    return df_norm, scaler

In [10]:
def feature_norm_transform(test_df, scaler):
    array =  test_df.values
    df_norm = pd.DataFrame(scaler.transform(array), columns=test_df.columns, index=test_df.index)
    return df_norm  

### Нормализация и сохранение скейлера для RDKit2D

Применяем нормализацию к отфильтрованным RDKit2D дескрипторам и сохраняем обученный `MinMaxScaler` с помощью `joblib`.


In [11]:
norm_train_rdkit2d, scaler_rdkit2d = feature_norm_fit(filtered_train_rdkit2d)

Проверка параметров scaler

In [12]:
scaler_rdkit2d.get_params

<bound method BaseEstimator.get_params of MinMaxScaler()>

Просмотр первой строки нормализованных данных

In [ ]:
norm_train_rdkit2d.head(1)

,BalabanJ,BertzCT,Chi0,EState_VSA1,EState_VSA10,EState_VSA11,EState_VSA2,EState_VSA3,EState_VSA4,EState_VSA5,...,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,qed
CASRN,,,,,,,,,,,,,,,,,,,,,
100-00-5,0.36578,0.013096,0.024088,0.00495,0.024023,0.0,0.026235,0.016894,0.000321,0.094796,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.48663


Экспорт scaler в файл

In [14]:
joblib.dump(scaler_rdkit2d, '../data/descriptors/scaler_rdkit2d.pkl') 

['../data/descriptors/scaler_rdkit2d.pkl']

Проверка работоспособности scaler 

Загрузка scaler

In [15]:
test_scaler = joblib.load('../data/descriptors/scaler_rdkit2d.pkl')

Тестирование на тренировочных данных

In [16]:
test_rdkit2d = feature_norm_transform(filtered_train_rdkit2d, test_scaler)

Сравнение с оригиналом

In [17]:
test_rdkit2d.head(1) 

,BalabanJ,BertzCT,Chi0,EState_VSA1,EState_VSA10,EState_VSA11,EState_VSA2,EState_VSA3,EState_VSA4,EState_VSA5,...,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,qed
CASRN,,,,,,,,,,,,,,,,,,,,,
100-00-5,0.36578,0.013096,0.024088,0.00495,0.024023,0.0,0.026235,0.016894,0.000321,0.094796,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.48663


Обработка Mordred дескрипторов

Нормализация тренировочных данных

Сохранение scaler

In [18]:
norm_train_mordred, scaler_mordred = feature_norm_fit(filtered_train_mordred)
joblib.dump(scaler_mordred, '../data/descriptors/scaler_mordred.pkl')

['../data/descriptors/scaler_mordred.pkl']

In [19]:
test_scaler_mordred = joblib.load('../data/descriptors/scaler_mordred.pkl')

In [20]:
test_mordred = feature_norm_transform(filtered_train_mordred, test_scaler_mordred)
test_mordred.head(1)

,nAcid,nBase,SpAbs_A,SpMax_A,SpMAD_A,LogEE_A,VE1_A,VR1_A,nAromAtom,nAtom,...,JGI5,JGI6,JGI7,JGI8,JGI9,JGI10,JGT10,SRW03,SRW05,TSRW10
CASRN,,,,,,,,,,,,,,,,,,,,,
100-00-5,0.0,0.0,0.023293,0.690983,0.836553,0.402493,0.269199,1.975149e-16,0.113208,0.013784,...,0.62963,0.0,0.0,0.0,0.0,0.0,0.533316,0.0,0.0,0.078559


In [21]:
norm_train_mordred.head(1)

,nAcid,nBase,SpAbs_A,SpMax_A,SpMAD_A,LogEE_A,VE1_A,VR1_A,nAromAtom,nAtom,...,JGI5,JGI6,JGI7,JGI8,JGI9,JGI10,JGT10,SRW03,SRW05,TSRW10
CASRN,,,,,,,,,,,,,,,,,,,,,
100-00-5,0.0,0.0,0.023293,0.690983,0.836553,0.402493,0.269199,1.975149e-16,0.113208,0.013784,...,0.62963,0.0,0.0,0.0,0.0,0.0,0.533316,0.0,0.0,0.078559


### Экспорт подготовленных данных

Сохранение тренировочных данных

In [22]:
filtered_train_ecfp6_bits.to_csv('../data/Bmodel_features/modeling_train_ecfp6_bits.csv')
filtered_train_ecfp6_counts.to_csv('../data/Bmodel_features/modeling_train_ecfp6_counts.csv')
filtered_train_maccs.to_csv('../data/Bmodel_features/modeling_train_maccs.csv')
norm_train_rdkit2d.to_csv('../data/Bmodel_features/modeling_train_rdkit2d.csv')
norm_train_mordred.to_csv('../data/Bmodel_features/modeling_train_mordred.csv')

### Подготовка тестовых данных

Функция обработки тестовых данных

In [ ]:
def test_feature(df, feature, scaler = None):
    
    with open('../data/descriptors/filtered_features.json') as f:
        dict_features = json.load(f)
        
    if feature not in dict_features.keys():
        raise Exception(f'The feature **{feature}** is not support, please choose from [ecfp6_bits, ecfp6_counts, maccs, rdkit2d, mordred]')
        
    filtered_desc = dict_features[feature]
    df = df[filtered_desc]
    
    if scaler:
        df = feature_norm_transform(df, scaler)
    
    return df

Загрузка тестовых данных

In [ ]:
test_ecfp6_bits = pd.read_csv('../data/descriptors/test_ecfp6_bits.csv', index_col='CASRN')
test_ecfp6_counts = pd.read_csv('../data/descriptors/test_ecfp6_counts.csv', index_col='CASRN')
test_maccs = pd.read_csv('../data/descriptors/test_maccs.csv', index_col='CASRN')
test_rdkit2d = pd.read_csv('../data/descriptors/test_rdkit2d.csv', index_col='CASRN')
test_mordred = pd.read_csv('../data/descriptors/test_mordred.csv', index_col='CASRN')

### Финализация тестовых данных

Обработка каждого типа данных

In [25]:
filtered_test_ecfp6_bits = test_feature(test_ecfp6_bits, 'ecfp6_bits')
filtered_test_ecfp6_counts = test_feature(test_ecfp6_counts, 'ecfp6_counts')
filtered_test_maccs = test_feature(test_maccs, 'maccs')

In [26]:
filtered_test_ecfp6_bits.shape, filtered_test_ecfp6_counts.shape, filtered_test_maccs.shape

((2895, 2048), (2895, 1584), (2895, 148))

In [27]:
scaler_mordred = joblib.load('../data/descriptors/scaler_mordred.pkl')
scaler_rdkit2d = joblib.load('../data/descriptors/scaler_rdkit2d.pkl') 

In [28]:
filtered_test_rdkit2d = test_feature(test_rdkit2d, 'rdkit2d', scaler_rdkit2d)

In [29]:
filtered_test_rdkit2d.head(1)

,BalabanJ,BertzCT,Chi0,EState_VSA1,EState_VSA10,EState_VSA11,EState_VSA2,EState_VSA3,EState_VSA4,EState_VSA5,...,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,qed
CASRN,,,,,,,,,,,,,,,,,,,,,
130209-82-4,0.207313,0.035201,0.072131,0.018411,0.047773,0.0,0.110288,0.129578,0.100139,0.021734,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.012987,0.0,0.255839


In [30]:
filtered_test_mordred = test_feature(test_mordred, 'mordred', scaler_mordred)

In [31]:
filtered_test_mordred.head(1)


,nAcid,nBase,SpAbs_A,SpMax_A,SpMAD_A,LogEE_A,VE1_A,VR1_A,nAromAtom,nAtom,...,JGI5,JGI6,JGI7,JGI8,JGI9,JGI10,JGT10,SRW03,SRW05,TSRW10
CASRN,,,,,,,,,,,,,,,,,,,,,
130209-82-4,0.0,0.0,0.074844,0.743328,0.867067,0.582633,0.335865,1.565354e-14,0.113208,0.085213,...,0.196928,0.252108,0.1689,0.266486,0.273758,0.249418,0.412305,0.0,0.415476,0.162592


In [32]:
filtered_test_mordred.shape, filtered_test_rdkit2d.shape

((2895, 446), (2895, 164))

Сохранение результатов

In [33]:
filtered_test_ecfp6_bits.to_csv('../data/Bmodel_features/modeling_test_ecfp6_bits.csv')
filtered_test_ecfp6_counts.to_csv('../data/Bmodel_features/modeling_test_ecfp6_counts.csv')
filtered_test_maccs.to_csv('../data/Bmodel_features/modeling_test_maccs.csv')
filtered_test_rdkit2d.to_csv('../data/Bmodel_features/modeling_test_rdkit2d.csv')
filtered_test_mordred.to_csv('../data/Bmodel_features/modeling_test_mordred.csv')